## Importing necessary library

In [64]:
import os
from google.cloud import language_v1
from google.cloud.language_v1 import enums
import matplotlib.pyplot as plt
import cv2
import argparse
import io
import json
from google.cloud import language
import numpy
import six

## Setting path to json key

In [65]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/affine/GCP/downloaded_key.json"

- Entity Analysis provides information about entities in the text, which generally refer to named "things" such as famous individuals, landmarks, common objects, etc.


- Entity analysis is performed with the analyzeEntities method.

# Analyzing Entities

## 1) Analyzing Entities in a String, text file and pdf file stored in local

In [66]:
from google.cloud import language_v1
from google.cloud.language_v1 import enums


def sample_analyze_entities(text_content):
    """
    Analyzing Entities in a String

    Args:
      text_content The text content to analyze
    """

    client = language_v1.LanguageServiceClient()

    # Available types: PLAIN_TEXT, HTML
    type_ = enums.Document.Type.PLAIN_TEXT

    # Optional. If not specified, the language is automatically detected.
    # For list of supported languages:
    # https://cloud.google.com/natural-language/docs/languages
    language = "en"
    document = {"content": text_content, "type": type_, "language": language}

    # Available values: NONE, UTF8, UTF16, UTF32
    encoding_type = enums.EncodingType.UTF8

    response = client.analyze_entities(document, encoding_type=encoding_type)
    print('response: \n',response)

    # Loop through entitites returned from the API
    for entity in response.entities:
        print(u"Representative name for the entity: {}".format(entity.name))

        # Get entity type, e.g. PERSON, LOCATION, ADDRESS, NUMBER, et al
        print(u"Entity type: {}".format(enums.Entity.Type(entity.type).name))

        # Get the salience score associated with the entity in the [0, 1.0] range
        print(u"Salience score: {}".format(entity.salience))

        # Loop over the metadata associated with entity. For many known entities,
        # the metadata is a Wikipedia URL (wikipedia_url) and Knowledge Graph MID (mid).
        # Some entity types may have additional metadata, e.g. ADDRESS entities
        # may have metadata for the address street_name, postal_code, et al.
        for metadata_name, metadata_value in entity.metadata.items():
            print(u"{}: {}".format(metadata_name, metadata_value))

        # Loop over the mentions of this entity in the input document.
        # The API currently supports proper noun mentions.
        for mention in entity.mentions:
            print(u"Mention text: {}".format(mention.text.content))

            # Get the mention type, e.g. PROPER for proper noun
            print(
                u"Mention type: {}".format(enums.EntityMention.Type(mention.type).name)
            )

    # Get the language of the text, which will be the same as
    # the language specified in the request or, if not specified,
    # the automatically-detected language.
    print(u"Language of the text: {}".format(response.language))


### a) Analyzing Entities in a String provided by user

Passing string to the above function

In [68]:
sample_analyze_entities("My name is Pavitra and I love my India.I like to visit Kerela once more")

response: 
 entities {
  name: "name"
  type: OTHER
  salience: 0.6281695365905762
  mentions {
    text {
      content: "name"
      begin_offset: 3
    }
    type: COMMON
  }
}
entities {
  name: "Pavitra"
  type: PERSON
  salience: 0.17903731763362885
  mentions {
    text {
      content: "Pavitra"
      begin_offset: 11
    }
    type: PROPER
  }
}
entities {
  name: "India."
  type: PERSON
  salience: 0.10441122204065323
  mentions {
    text {
      content: "India."
      begin_offset: 33
    }
    type: PROPER
  }
}
entities {
  name: "Kerela"
  type: LOCATION
  metadata {
    key: "mid"
    value: "/m/0byh8j"
  }
  metadata {
    key: "wikipedia_url"
    value: "https://en.wikipedia.org/wiki/Kerala"
  }
  salience: 0.08838191628456116
  mentions {
    text {
      content: "Kerela"
      begin_offset: 55
    }
    type: PROPER
  }
}
language: "en"

Representative name for the entity: name
Entity type: OTHER
Salience score: 0.6281695365905762
Mention text: name
Mention type: 

** Output**

Response of Entity analysis returns contains:

- a set of detected entities, 

- parameters associated with those entities:

    - the entity's type
    - metadata : contains source information about the entity's knowledge repository
    - salience: relevance of the entity to the overall text
    - mentions: locations in the text that refer to the same entity. 
    
    
    
- Entities are returned in the order (highest to lowest) of their salience scores, which reflect their relevance to the overall text.

###  b) Analyzing Entity in a text file stored in local

First reading text file and then passing text content to analyze entities on it.

In [69]:
# reading text file
filename = r'/home/affine/GCP/NLP_Entity_Extraction/Entity2.txt'
file=open(filename,'r')
text_content=file.read()
print(text_content)


# calling function to analyze sentiment
sample_analyze_entities(text_content)

A microwave oven (commonly referred to as a microwave) is an electric oven that heats and cooks food by exposing it to electromagnetic radiation in the microwave frequency range.American engineer Percy Spencer is generally credited with inventing the modern microwave oven after World War II from radar technology developed during the war. Named the "Radarange", it was first sold in 1946.

response: 
 entities {
  name: "microwave oven"
  type: CONSUMER_GOOD
  salience: 0.5929460525512695
  mentions {
    text {
      content: "microwave oven"
      begin_offset: 2
    }
    type: COMMON
  }
  mentions {
    text {
      content: "oven"
      begin_offset: 70
    }
    type: COMMON
  }
}
entities {
  name: "microwave oven"
  type: CONSUMER_GOOD
  salience: 0.13975511491298676
  mentions {
    text {
      content: "microwave oven"
      begin_offset: 258
    }
    type: COMMON
  }
}
entities {
  name: "Percy Spencer"
  type: PERSON
  metadata {
    key: "mid"
    value: "/m/01ryz3"
  }
 

### c) Analyzing Entity in a pdf file stored in local

First extracting text from pdf file using cloud vision api and then analyzing entities on extracted text.

In [70]:
from google.cloud import vision_v1
from google.cloud.vision_v1 import enums as e2

def analyze_entities(file_path):
    client = vision_v1.ImageAnnotatorClient()
    mime_type = "application/pdf"
    with io.open(file_path, "rb") as f:
        content = f.read()
    input_config = {"mime_type": mime_type, "content": content}
    features = [{"type": e2.Feature.Type.DOCUMENT_TEXT_DETECTION}]
    requests = [{"input_config": input_config, "features": features}]
    response = client.batch_annotate_files(requests)
    for image_response in response.responses[0].responses:
        pdf_text=image_response.full_text_annotation.text
        print(pdf_text)
    
    # calling below function to analyze entities
    sample_analyze_entities(pdf_text)

In [71]:
analyze_entities('Entityp2.pdf')

Customers can purchase items from the comfort of their own homes or work place. Shopping is made easier and convenient for the customer through internet. It is also easy to cancel the transactions.Long duration and lack of proper inventory management result in delays in shipment.

response: 
 entities {
  name: "Customers"
  type: PERSON
  salience: 0.4366687536239624
  mentions {
    text {
      content: "Customers"
    }
    type: COMMON
  }
}
entities {
  name: "items"
  type: CONSUMER_GOOD
  salience: 0.1631091982126236
  mentions {
    text {
      content: "items"
      begin_offset: 23
    }
    type: COMMON
  }
}
entities {
  name: "comfort"
  type: OTHER
  salience: 0.1631091982126236
  mentions {
    text {
      content: "comfort"
      begin_offset: 38
    }
    type: COMMON
  }
}
entities {
  name: "homes"
  type: LOCATION
  salience: 0.06615269929170609
  mentions {
    text {
      content: "homes"
      begin_offset: 59
    }
    type: COMMON
  }
}
entities {
  name: "

## 2) Analyzing Entities from Google Cloud Storage

The Natural Language can perform entity analysis directly on a file located in Google Cloud Storage, without the need to send the contents of the file in the body of your request.

In [23]:
from google.cloud import language_v1
from google.cloud.language_v1 import enums


def sample_analyze_entities_uri(gcs_content_uri):
    """
    Analyzing Entities in text file stored in Cloud Storage

    Args:
      gcs_content_uri Google Cloud Storage URI where the file content is located.
      e.g. gs://[Your Bucket]/[Path to File]
    """

    client = language_v1.LanguageServiceClient()

    # gcs_content_uri = 'gs://cloud-samples-data/language/entity.txt'

    # Available types: PLAIN_TEXT, HTML
    type_ = enums.Document.Type.PLAIN_TEXT

    # Optional. If not specified, the language is automatically detected.
    # For list of supported languages:
    # https://cloud.google.com/natural-language/docs/languages
    language = "en"
    document = {"gcs_content_uri": gcs_content_uri, "type": type_, "language": language}

    # Available values: NONE, UTF8, UTF16, UTF32
    encoding_type = enums.EncodingType.UTF8

    response = client.analyze_entities(document, encoding_type=encoding_type)
    print("response: \n", response)
    
    # Loop through entitites returned from the API
    for entity in response.entities:
        print(u"Representative name for the entity: {}".format(entity.name))
        # Get entity type, e.g. PERSON, LOCATION, ADDRESS, NUMBER, et al
        print(u"Entity type: {}".format(enums.Entity.Type(entity.type).name))
        # Get the salience score associated with the entity in the [0, 1.0] range
        print(u"Salience score: {}".format(entity.salience))
        # Loop over the metadata associated with entity. For many known entities,
        # the metadata is a Wikipedia URL (wikipedia_url) and Knowledge Graph MID (mid).
        # Some entity types may have additional metadata, e.g. ADDRESS entities
        # may have metadata for the address street_name, postal_code, et al.
        for metadata_name, metadata_value in entity.metadata.items():
            print(u"{}: {}".format(metadata_name, metadata_value))

        # Loop over the mentions of this entity in the input document.
        # The API currently supports proper noun mentions.
        for mention in entity.mentions:
            print(u"Mention text: {}".format(mention.text.content))
            # Get the mention type, e.g. PROPER for proper noun
            print(
                u"Mention type: {}".format(enums.EntityMention.Type(mention.type).name)
            )

    # Get the language of the text, which will be the same as
    # the language specified in the request or, if not specified,
    # the automatically-detected language.
    print(u"Language of the text: {}".format(response.language))


### a) Analyzing Entities from text file stored in Google Cloud Storage

Text used for below example:

Donald John Trump (born June 14, 1946) is the 45th and current president of the United States.Trump's political positions have been described as populist, protectionist, and nationalist.Trump began his career in 1968 at his father Fred's real estate development company, E. Trump & Son.

In [24]:
sample_analyze_entities_uri("gs://bucket0406/Entity1.txt")

response: 
 entities {
  name: "United States.Trump"
  type: PERSON
  salience: 0.34874916076660156
  mentions {
    text {
      content: "United States.Trump"
      begin_offset: 80
    }
    type: PROPER
  }
  mentions {
    text {
      content: "Trump"
      begin_offset: 186
    }
    type: PROPER
  }
}
entities {
  name: "Donald John Trump"
  type: PERSON
  metadata {
    key: "mid"
    value: "/m/0cqt90"
  }
  metadata {
    key: "wikipedia_url"
    value: "https://en.wikipedia.org/wiki/Donald_Trump"
  }
  salience: 0.2355983406305313
  mentions {
    text {
      content: "Donald John Trump"
    }
    type: PROPER
  }
}
entities {
  name: "Fred"
  type: PERSON
  salience: 0.10649247467517853
  mentions {
    text {
      content: "father"
      begin_offset: 224
    }
    type: COMMON
  }
  mentions {
    text {
      content: "Fred"
      begin_offset: 231
    }
    type: PROPER
  }
}
entities {
  name: "E. Trump & Son"
  type: ORGANIZATION
  metadata {
    key: "mid"
    val

### b) Analyzing Entities from pdf file stored in Google Cloud Storage

For this first we extract text from pdf file using cloud vision api and then analyze entities on that text.

In [72]:
from google.cloud import vision_v1
from google.cloud.vision_v1 import enums as e1
def analyze_entity_pdf(storage_uri,):
    mime_type = "application/pdf"
    client = vision_v1.ImageAnnotatorClient()
    gcs_source = {"uri": storage_uri}
    input_config = {"gcs_source": gcs_source, "mime_type": mime_type}
    features = [{"type": e1.Feature.Type.DOCUMENT_TEXT_DETECTION}]
    requests = [{"input_config": input_config, "features": features}]
    response = client.batch_annotate_files(requests)
    for image_response in response.responses[0].responses:
        pdf_text=image_response.full_text_annotation.text
        print(pdf_text)
        
    # calling below function to analyze entities on extracted text    
    sample_analyze_entities(pdf_text)

In [73]:
analyze_entity_pdf("gs://bucket0406/Entityp1.pdf")

A microwave oven (commonly referred to as a microwave) is an electric oven that heats and cooks food by exposing it to electromagnetic radiation in the microwave frequency range.American engineer Percy Spencer is generally credited with inventing the modern microwave oven after World War II from radar technology developed during the war. Named the "Radarange", it was first sold in 1946.

response: 
 entities {
  name: "microwave oven"
  type: CONSUMER_GOOD
  salience: 0.5929460525512695
  mentions {
    text {
      content: "microwave oven"
      begin_offset: 2
    }
    type: COMMON
  }
  mentions {
    text {
      content: "oven"
      begin_offset: 70
    }
    type: COMMON
  }
}
entities {
  name: "microwave oven"
  type: CONSUMER_GOOD
  salience: 0.13975511491298676
  mentions {
    text {
      content: "microwave oven"
      begin_offset: 258
    }
    type: COMMON
  }
}
entities {
  name: "Percy Spencer"
  type: PERSON
  metadata {
    key: "mid"
    value: "/m/01ryz3"
  }
 

# Analyzing Entity Sentiment

- Entity Sentiment Analysis combines both entity analysis and sentiment analysis and attempts to determine the sentiment (positive or negative) expressed about entities within the text. 


- Entity sentiment is represented by numerical score and magnitude values and is determined for each mention of an entity. Those scores are then aggregated into an overall sentiment score and magnitude for an entity.


- Entity Sentiment Analysis requests are sent to the Natural Language through use of the analyzeEntitySentiment.

## 1) Analyzing Entity Sentiment on string, text file and pdf file stored in local

In [74]:
from google.cloud import language_v1
from google.cloud.language_v1 import enums


def sample_analyze_entity_sentiment(text_content):
    """
    Analyzing Entity Sentiment in a String

    Args:
      text_content The text content to analyze
    """

    client = language_v1.LanguageServiceClient()

    # Available types: PLAIN_TEXT, HTML
    type_ = enums.Document.Type.PLAIN_TEXT

    # Optional. If not specified, the language is automatically detected.
    # For list of supported languages:
    # https://cloud.google.com/natural-language/docs/languages
    language = "en"
    document = {"content": text_content, "type": type_, "language": language}

    # Available values: NONE, UTF8, UTF16, UTF32
    encoding_type = enums.EncodingType.UTF8

    response = client.analyze_entity_sentiment(document, encoding_type=encoding_type)
    print("response: \n",response)
    
    # Loop through entitites returned from the API
    for entity in response.entities:
        print(u"Representative name for the entity: {}".format(entity.name))
        # Get entity type, e.g. PERSON, LOCATION, ADDRESS, NUMBER, et al
        print(u"Entity type: {}".format(enums.Entity.Type(entity.type).name))
        # Get the salience score associated with the entity in the [0, 1.0] range
        print(u"Salience score: {}".format(entity.salience))
        # Get the aggregate sentiment expressed for this entity in the provided document.
        sentiment = entity.sentiment
        print(u"Entity sentiment score: {}".format(sentiment.score))
        print(u"Entity sentiment magnitude: {}".format(sentiment.magnitude))
        # Loop over the metadata associated with entity. For many known entities,
        # the metadata is a Wikipedia URL (wikipedia_url) and Knowledge Graph MID (mid).
        # Some entity types may have additional metadata, e.g. ADDRESS entities
        # may have metadata for the address street_name, postal_code, et al.
        for metadata_name, metadata_value in entity.metadata.items():
            print(u"{} = {}".format(metadata_name, metadata_value))

        # Loop over the mentions of this entity in the input document.
        # The API currently supports proper noun mentions.
        for mention in entity.mentions:
            print(u"Mention text: {}".format(mention.text.content))
            # Get the mention type, e.g. PROPER for proper noun
            print(
                u"Mention type: {}".format(enums.EntityMention.Type(mention.type).name)
            )

    # Get the language of the text, which will be the same as
    # the language specified in the request or, if not specified,
    # the automatically-detected language.
    print(u"Language of the text: {}".format(response.language))


### a) Analyzing entity sentiment provided a string

In [45]:
sample_analyze_entity_sentiment("Rohan behaves very nicely to me. His behaviour is not good to others.He is bad guy for other people")

response: 
 entities {
  name: "Rohan"
  type: PERSON
  salience: 0.9308645725250244
  mentions {
    text {
      content: "Rohan"
    }
    type: PROPER
    sentiment {
      magnitude: 0.8999999761581421
      score: 0.8999999761581421
    }
  }
  mentions {
    text {
      content: "guy"
      begin_offset: 79
    }
    type: COMMON
    sentiment {
      magnitude: 0.8999999761581421
      score: -0.8999999761581421
    }
  }
  sentiment {
    magnitude: 3.299999952316284
    score: -0.30000001192092896
  }
}
entities {
  name: "behaviour"
  type: OTHER
  salience: 0.04898858070373535
  mentions {
    text {
      content: "behaviour"
      begin_offset: 37
    }
    type: COMMON
    sentiment {
      magnitude: 0.8999999761581421
      score: -0.8999999761581421
    }
  }
  sentiment {
    magnitude: 0.8999999761581421
    score: -0.8999999761581421
  }
}
entities {
  name: "people"
  type: PERSON
  salience: 0.02014683187007904
  mentions {
    text {
      content: "people"
   

** Output **

The Natural Language API processes the given text to extract the entities and determine sentiment.

An Entity Sentiment Analysis request returns a response containing:

- entities that were found in the document content

- a mentions entry for each time the entity is mentioned

- the numerical score and magnitude values for each mention.

- The overall score and magnitude values for an entity are an aggregate of the specific score and magnitude values for each mention of the entity. 

- The score and magnitude values for an entity can be 0, if there was low sentiment in the text, resulting in a magnitude of 0, or the sentiment is mixed, resulting in a score of 0.

###  b) Analyzing entity sentiment on text file stored in local

In [47]:
# reading text file
filename = r'/home/affine/GCP/NLP_Entity_Extraction/Entity2.txt'
file=open(filename,'r')
text_content=file.read()
print(text_content)

# calling function to analyze sentiment
sample_analyze_entity_sentiment(text_content)

A microwave oven (commonly referred to as a microwave) is an electric oven that heats and cooks food by exposing it to electromagnetic radiation in the microwave frequency range.American engineer Percy Spencer is generally credited with inventing the modern microwave oven after World War II from radar technology developed during the war. Named the "Radarange", it was first sold in 1946.

response: 
 entities {
  name: "microwave oven"
  type: CONSUMER_GOOD
  salience: 0.5929460525512695
  mentions {
    text {
      content: "microwave oven"
      begin_offset: 2
    }
    type: COMMON
    sentiment {
      magnitude: 0.10000000149011612
      score: -0.10000000149011612
    }
  }
  mentions {
    text {
      content: "oven"
      begin_offset: 70
    }
    type: COMMON
    sentiment {
    }
  }
  sentiment {
    magnitude: 0.4000000059604645
    score: -0.10000000149011612
  }
}
entities {
  name: "microwave oven"
  type: CONSUMER_GOOD
  salience: 0.13975511491298676
  mentions {
   

## c) Analyzing entity sentiment on pdf file stored in local

In [49]:
from google.cloud import vision_v1
from google.cloud.vision_v1 import enums as e2

def analyze_entities(file_path):
    client = vision_v1.ImageAnnotatorClient()
    mime_type = "application/pdf"
    with io.open(file_path, "rb") as f:
        content = f.read()
    input_config = {"mime_type": mime_type, "content": content}
    features = [{"type": e2.Feature.Type.DOCUMENT_TEXT_DETECTION}]
    requests = [{"input_config": input_config, "features": features}]
    response = client.batch_annotate_files(requests)
    for image_response in response.responses[0].responses:
        pdf_text=image_response.full_text_annotation.text
        print(pdf_text)
    
    # calling below function to analyze sentiment
    sample_analyze_entity_sentiment(pdf_text)

In [51]:
analyze_entities('Entityp2.pdf')

Customers can purchase items from the comfort of their own homes or work place. Shopping is made easier and convenient for the customer through internet. It is also easy to cancel the transactions.Long duration and lack of proper inventory management result in delays in shipment.

response: 
 entities {
  name: "Customers"
  type: PERSON
  salience: 0.4366687536239624
  mentions {
    text {
      content: "Customers"
    }
    type: COMMON
    sentiment {
    }
  }
  sentiment {
  }
}
entities {
  name: "items"
  type: CONSUMER_GOOD
  salience: 0.1631091982126236
  mentions {
    text {
      content: "items"
      begin_offset: 23
    }
    type: COMMON
    sentiment {
    }
  }
  sentiment {
  }
}
entities {
  name: "comfort"
  type: OTHER
  salience: 0.1631091982126236
  mentions {
    text {
      content: "comfort"
      begin_offset: 38
    }
    type: COMMON
    sentiment {
      magnitude: 0.10000000149011612
      score: 0.10000000149011612
    }
  }
  sentiment {
    magnitu

## 2) Analyzing Entity Sentiment from Google Cloud Storage

In [60]:
from google.cloud import language_v1
from google.cloud.language_v1 import enums


def sample_analyze_entity_sentiment_uri(gcs_content_uri):
    """
    Analyzing Entity Sentiment in text file stored in Cloud Storage

    Args:
      gcs_content_uri Google Cloud Storage URI where the file content is located.
      e.g. gs://[Your Bucket]/[Path to File]
    """

    client = language_v1.LanguageServiceClient()

    # gcs_content_uri = 'gs://cloud-samples-data/language/entity-sentiment.txt'

    # Available types: PLAIN_TEXT, HTML
    type_ = enums.Document.Type.PLAIN_TEXT

    # Optional. If not specified, the language is automatically detected.
    # For list of supported languages:
    # https://cloud.google.com/natural-language/docs/languages
    language = "en"
    document = {"gcs_content_uri": gcs_content_uri, "type": type_, "language": language}

    # Available values: NONE, UTF8, UTF16, UTF32
    encoding_type = enums.EncodingType.UTF8

    response = client.analyze_entity_sentiment(document, encoding_type=encoding_type)
    # Loop through entitites returned from the API
    for entity in response.entities:
        print(u"Representative name for the entity: {}".format(entity.name))
        # Get entity type, e.g. PERSON, LOCATION, ADDRESS, NUMBER, et al
        print(u"Entity type: {}".format(enums.Entity.Type(entity.type).name))
        # Get the salience score associated with the entity in the [0, 1.0] range
        print(u"Salience score: {}".format(entity.salience))
        # Get the aggregate sentiment expressed for this entity in the provided document.
        sentiment = entity.sentiment
        print(u"Entity sentiment score: {}".format(sentiment.score))
        print(u"Entity sentiment magnitude: {}".format(sentiment.magnitude))
        # Loop over the metadata associated with entity. For many known entities,
        # the metadata is a Wikipedia URL (wikipedia_url) and Knowledge Graph MID (mid).
        # Some entity types may have additional metadata, e.g. ADDRESS entities
        # may have metadata for the address street_name, postal_code, et al.
        for metadata_name, metadata_value in entity.metadata.items():
            print(u"{} = {}".format(metadata_name, metadata_value))

        # Loop over the mentions of this entity in the input document.
        # The API currently supports proper noun mentions.
        for mention in entity.mentions:
            print(u"Mention text: {}".format(mention.text.content))
            # Get the mention type, e.g. PROPER for proper noun
            print(
                u"Mention type: {}".format(enums.EntityMention.Type(mention.type).name)
            )

    # Get the language of the text, which will be the same as
    # the language specified in the request or, if not specified,
    # the automatically-detected language.
    print(u"Language of the text: {}".format(response.language))


### a) Analyzing entity sentiment stored in a text file on Google Cloud Storage

In [53]:
sample_analyze_entity_sentiment("gs://bucket0406/Entity1.txt")

Representative name for the entity: United States.Trump
Entity type: PERSON
Salience score: 0.34874916076660156
Entity sentiment score: 0.0
Entity sentiment magnitude: 0.0
Mention text: United States.Trump
Mention type: PROPER
Mention text: Trump
Mention type: PROPER
Representative name for the entity: Donald John Trump
Entity type: PERSON
Salience score: 0.2355983406305313
Entity sentiment score: 0.0
Entity sentiment magnitude: 0.10000000149011612
wikipedia_url = https://en.wikipedia.org/wiki/Donald_Trump
mid = /m/0cqt90
Mention text: Donald John Trump
Mention type: PROPER
Representative name for the entity: Fred
Entity type: PERSON
Salience score: 0.10649247467517853
Entity sentiment score: 0.0
Entity sentiment magnitude: 0.0
Mention text: father
Mention type: COMMON
Mention text: Fred
Mention type: PROPER
Representative name for the entity: E. Trump & Son
Entity type: ORGANIZATION
Salience score: 0.08423490822315216
Entity sentiment score: 0.0
Entity sentiment magnitude: 0.100000001

### b) Analyzing entity sentiment stored in a pdf file on Google Cloud Storage

In [58]:
from google.cloud import vision_v1
from google.cloud.vision_v1 import enums as e1
def analyze_entity_pdf(storage_uri,):
    mime_type = "application/pdf"
    client = vision_v1.ImageAnnotatorClient()
    gcs_source = {"uri": storage_uri}
    input_config = {"gcs_source": gcs_source, "mime_type": mime_type}
    features = [{"type": e1.Feature.Type.DOCUMENT_TEXT_DETECTION}]
    requests = [{"input_config": input_config, "features": features}]
    response = client.batch_annotate_files(requests)
    for image_response in response.responses[0].responses:
        pdf_text=image_response.full_text_annotation.text
        print(pdf_text)
        
    # calling below function to analyze sentiment on extracted text    
    sample_analyze_entity_sentiment(pdf_text)

In [62]:
analyze_entity_pdf("gs://bucket0406/Entityp1.pdf")

A microwave oven (commonly referred to as a microwave) is an electric oven that heats and cooks food by exposing it to electromagnetic radiation in the microwave frequency range.American engineer Percy Spencer is generally credited with inventing the modern microwave oven after World War II from radar technology developed during the war. Named the "Radarange", it was first sold in 1946.

response: 
 entities {
  name: "microwave oven"
  type: CONSUMER_GOOD
  salience: 0.5929460525512695
  mentions {
    text {
      content: "microwave oven"
      begin_offset: 2
    }
    type: COMMON
    sentiment {
      magnitude: 0.10000000149011612
      score: -0.10000000149011612
    }
  }
  mentions {
    text {
      content: "oven"
      begin_offset: 70
    }
    type: COMMON
    sentiment {
    }
  }
  sentiment {
    magnitude: 0.4000000059604645
    score: -0.10000000149011612
  }
}
entities {
  name: "microwave oven"
  type: CONSUMER_GOOD
  salience: 0.13975511491298676
  mentions {
   